# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 10:17AM,239240,21,399290130,"NBTY Global, Inc.",Released
1,Jun 9 2022 10:16AM,239239,15,VT80173443,"Virtus Pharmaceuticals, LLC",Released
2,Jun 9 2022 10:16AM,239239,15,VT80173457,"Virtus Pharmaceuticals, LLC",Released
3,Jun 9 2022 10:16AM,239239,15,VT80173458,"Virtus Pharmaceuticals, LLC",Released
4,Jun 9 2022 10:16AM,239239,15,VT80173464,"Virtus Pharmaceuticals, LLC",Released
5,Jun 9 2022 10:16AM,239239,15,VT80173465,"Virtus Pharmaceuticals, LLC",Released
6,Jun 9 2022 10:15AM,239237,15,PNC366974A,"Person & Covey, Inc.",Released
7,Jun 9 2022 10:15AM,239237,15,PNC366976A,"Person & Covey, Inc.",Released
8,Jun 9 2022 10:15AM,239237,15,PNC366977A,"Person & Covey, Inc.",Released
9,Jun 9 2022 10:15AM,239235,15,8118896,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
64,239236,Released,1
65,239237,Released,3
66,239238,Released,1
67,239239,Released,5
68,239240,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239236,NaN,NaN,1.0
239237,NaN,NaN,3.0
239238,NaN,NaN,1.0
239239,NaN,NaN,5.0
239240,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239107,0.0,0.0,1.0
239110,0.0,1.0,0.0
239122,0.0,0.0,1.0
239134,0.0,14.0,2.0
239138,13.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239107,0,0,1
239110,0,1,0
239122,0,0,1
239134,0,14,2
239138,13,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239107,0,0,1
1,239110,0,1,0
2,239122,0,0,1
3,239134,0,14,2
4,239138,13,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239107,,,1
1,239110,,1,
2,239122,,,1
3,239134,,14,2
4,239138,13,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc."
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC"
6,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc."
9,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC"
11,Jun 9 2022 10:10AM,239238,16,American International Chemical
12,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC"
20,Jun 9 2022 10:06AM,239233,15,"Chadwick Pharmaceuticals, Inc."
22,Jun 9 2022 10:06AM,239232,15,"Brookfield Pharmaceuticals, LLC"
23,Jun 9 2022 10:04AM,239236,21,"NBTY Global, Inc."
24,Jun 9 2022 10:02AM,239231,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",,,1
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",,,5
2,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",,,3
3,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",,,2
4,Jun 9 2022 10:10AM,239238,16,American International Chemical,,,1
5,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC",,,8
6,Jun 9 2022 10:06AM,239233,15,"Chadwick Pharmaceuticals, Inc.",,,2
7,Jun 9 2022 10:06AM,239232,15,"Brookfield Pharmaceuticals, LLC",,,1
8,Jun 9 2022 10:04AM,239236,21,"NBTY Global, Inc.",,,1
9,Jun 9 2022 10:02AM,239231,15,"Alliance Pharma, Inc.",,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",1,,
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",5,,
2,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",3,,
3,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",2,,
4,Jun 9 2022 10:10AM,239238,16,American International Chemical,1,,
5,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC",8,,
6,Jun 9 2022 10:06AM,239233,15,"Chadwick Pharmaceuticals, Inc.",2,,
7,Jun 9 2022 10:06AM,239232,15,"Brookfield Pharmaceuticals, LLC",1,,
8,Jun 9 2022 10:04AM,239236,21,"NBTY Global, Inc.",1,,
9,Jun 9 2022 10:02AM,239231,15,"Alliance Pharma, Inc.",19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",1,,
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",5,,
2,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",3,,
3,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",2,,
4,Jun 9 2022 10:10AM,239238,16,American International Chemical,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",5.0,NaN,NaN
2,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",3.0,NaN,NaN
3,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",2.0,NaN,NaN
4,Jun 9 2022 10:10AM,239238,16,American International Chemical,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",5.0,0.0,0.0
2,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",3.0,0.0,0.0
3,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",2.0,0.0,0.0
4,Jun 9 2022 10:10AM,239238,16,American International Chemical,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5262376,159.0,43.0,26.0
12,239221,3.0,11.0,0.0
15,1674641,40.0,0.0,0.0
16,717508,2.0,1.0,0.0
18,956703,4.0,0.0,0.0
19,1913463,30.0,26.0,35.0
20,956792,13.0,17.0,0.0
21,1196017,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5262376,159.0,43.0,26.0
1,12,239221,3.0,11.0,0.0
2,15,1674641,40.0,0.0,0.0
3,16,717508,2.0,1.0,0.0
4,18,956703,4.0,0.0,0.0
5,19,1913463,30.0,26.0,35.0
6,20,956792,13.0,17.0,0.0
7,21,1196017,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,159.0,43.0,26.0
1,12,3.0,11.0,0.0
2,15,40.0,0.0,0.0
3,16,2.0,1.0,0.0
4,18,4.0,0.0,0.0
5,19,30.0,26.0,35.0
6,20,13.0,17.0,0.0
7,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,159.0
1,12,Released,3.0
2,15,Released,40.0
3,16,Released,2.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,26.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
Executing,43.0,11.0,0.0,1.0,0.0,26.0,17.0,0.0
Released,159.0,3.0,40.0,2.0,4.0,30.0,13.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
1,Executing,43.0,11.0,0.0,1.0,0.0,26.0,17.0,0.0
2,Released,159.0,3.0,40.0,2.0,4.0,30.0,13.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
1,Executing,43.0,11.0,0.0,1.0,0.0,26.0,17.0,0.0
2,Released,159.0,3.0,40.0,2.0,4.0,30.0,13.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()